In [19]:
import numpy as np
from sklearn.cross_validation import train_test_split


def get_data():
    with open("data/ham_data.txt", encoding="utf8") as ham_f, open("data/spam_data.txt", encoding="utf8") as spam_f:
        ham_data = ham_f.readlines()
        spam_data = spam_f.readlines()
        ham_label = np.ones(len(ham_data)).tolist()
        spam_label = np.zeros(len(spam_data)).tolist()
        corpus = ham_data + spam_data
        labels = ham_label + spam_label
    return corpus, labels


def prepare_datasets(corpus, labels, test_data_proportion=0.3):
    train_X, test_X, train_Y, test_Y = train_test_split(corpus, labels,test_size=test_data_proportion, random_state=42)
    return train_X, test_X, train_Y, test_Y
corpus, labels=get_data()
train_corpus, test_corpus, train_labels, test_labels = prepare_datasets(corpus,
                                                                            labels,
                                                                    test_data_proportion=0.3)


In [20]:
import re
import string
import jieba

# 加载停用词
with open("dict/stop_words.utf8", encoding="utf8") as f:
    stopword_list = f.readlines()


def tokenize_text(text):
    tokens = jieba.cut(text)
    tokens = [token.strip() for token in tokens]
    return tokens


def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


def normalize_corpus(corpus, tokenize=False):
    normalized_corpus = []
    for text in corpus:

        text = remove_special_characters(text)
        text = remove_stopwords(text)
        normalized_corpus.append(text)
        if tokenize:
            text = tokenize_text(text)
            normalized_corpus.append(text)

    return normalized_corpus

In [21]:
norm_train_corpus = normalize_corpus(train_corpus)
norm_test_corpus = normalize_corpus(test_corpus)
norm_test_corpus

['中信  （  国际  ）  电子科技  有限公司  推出  新  产品  ：  升职  步步高  、  做生意  发大财  、  连  找  情人  都  用  的  上  ，  详情  进入  网  址  http  www  usa5588  com  ccc  电话  ：  020  33770208  服务  热线  ：  013650852999',
 '分  专业  吧  ，  也  分  导师  吧  标  题  Re  问  一个  ：  有人  觉得  自己  博士  能  混  毕业  吗  当然  很  好  混  毕业  了  博士  读 到  快  中期  了  ，  始终  感觉  什么  都  不会  ，  文章  也  没发  几篇  好  的  ，  论文  的  架构  也  没有  ，  一切  跟  刚  上  的  时候  没有  区别  。  但是  事实上  我  也  很  辛苦  的  找  资料  ，  做  实验  ，  还进  公司  实习  过  ，  现在  感觉  好  失败  ，  内心  已经  放弃  了  ，  打算  混  毕业  ，  不  知道  过来人  有  什么  高招  ，  请  指点  一二  。',
 '三年  的  水木  生活  版大  一直  都  照着  我  我  正是  在  伊  的  关照  下  逐步  逐步  的  迈入  水车  行列  可惜  我  刚 来  贵版  伊  就要  走  了  留个  小整  以作  纪念  版大  真是  个  好人  恩  robertwc  我  不  想要  nick  共上站  1289  次  ，  发表  过  48999  篇文章  上次  在  Wed  Sep  7  14  26  34  2005  从  211151  9088  到  本站  一游  。  离线  时间  因  在线  上  或  非常  断线  不详  信箱  ：  生命力  ：  身份  版主  。  目前  在  站上  ，  状态  如下  ：',
 '谢谢  你  可能  是  这个  浮躁  的  社会  促使  人  也  浮躁  道德  丧失  ，  所以  现在  的  婚姻  失败  的  很多  。  

In [22]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
V=TfidfVectorizer(ngram_range=(1,2),use_idf=True,smooth_idf=True)
M=V.fit_transform(norm_train_corpus)

In [23]:
feature_names = V.get_feature_names()

In [24]:
from sklearn.cluster import KMeans


def k_means(feature_matrix, num_clusters=10):
    km = KMeans(n_clusters=num_clusters,
                max_iter=10000)
    km.fit(feature_matrix)
    clusters = km.labels_
    return km, clusters


num_clusters = 10
km_obj, clusters = k_means(feature_matrix=M,
                           num_clusters=num_clusters)


In [25]:
def get_cluster_data(clustering_obj, data,
                     feature_names, num_clusters,
                     topn_features=10):
    cluster_details = {}
    # 获取cluster的center
    ordered_centroids = clustering_obj.cluster_centers_.argsort()[:, ::-1]
    # 获取每个cluster的关键特征
    # 获取每个cluster的书
    for cluster_num in range(num_clusters):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster_num'] = cluster_num
        key_features = [feature_names[index]
                        for index
                        in ordered_centroids[cluster_num, :topn_features]]
        cluster_details[cluster_num]['key_features'] = key_features

    return cluster_details

In [26]:
def print_cluster_data(cluster_data):
    # print cluster details
    for cluster_num, cluster_details in cluster_data.items():
        print('Cluster {} details:'.format(cluster_num))
        print('-' * 20)
        print('Key features:', cluster_details['key_features'])

        print('=' * 40)

In [27]:
cluster_data = get_cluster_data(clustering_obj=km_obj,
                                data=norm_train_corpus,
                                feature_names=feature_names,
                                num_clusters=num_clusters,
                                topn_features=5)

print_cluster_data(cluster_data)

Cluster 0 details:
--------------------
Key features: ['仔细 固定', '依靠 网络', '时间 仔细', '赚钱 生意', '眼光 极度']
Cluster 1 details:
--------------------
Key features: ['详情 进入', '步步高', 'com ccc', '产品 升职', '热线 013650852999']
Cluster 2 details:
--------------------
Key features: ['注册', '广告', '自己 电子名片', '电子名片', '服务']
Cluster 3 details:
--------------------
Key features: ['iframe http', 'iframe', '请点此', '显示', '请点此 iframe']
Cluster 4 details:
--------------------
Key features: ['航空', '特价 超市', '下载 网上支付', '航空 代理', '蓝天 下载']
Cluster 5 details:
--------------------
Key features: ['公司', '缴款 如果', '外代 国税', '广告 海关', '如果 公司']
Cluster 6 details:
--------------------
Key features: ['一个', '自己', '没有', 'http', 'com']
Cluster 7 details:
--------------------
Key features: ['免费', 'com', '网站', '下载 终身', 'http cnysoft']
Cluster 8 details:
--------------------
Key features: ['知道', '女友', '晚餐', '迷恋', '休息']
Cluster 9 details:
--------------------
Key features: ['公司', '发票', '代开', '联系人', '来电']
